## LCEL

通过对LCEL的各种Primitives组成，可以有多种使用技巧

In [1]:
# 环境变量设置
import os
os.environ["OPENAI_API_KEY"] = "sk-xxx"
os.environ["OPENAI_API_BASE"] = "https://api.chatanywhere.tech/v1"

### Runnables

#### 使用 Runnables来连接多条链

In [6]:
#获取可迭代对象中指定索引或键对应的元素
from operator import itemgetter 

from langchain.schema import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt1 = ChatPromptTemplate.from_template("{person}出生于哪个城市?")
prompt2 = ChatPromptTemplate.from_template(
    "{city}属于哪个省和哪个城市? 用{language}来回答"
)

model = ChatOpenAI(model="gpt-4")

chain1 = prompt1 | model | StrOutputParser()

chain2 = (
    {"city": chain1, "language": itemgetter("language")} #获取invoke中的language
    | prompt2
    | model
    | StrOutputParser()
)
# chain1.invoke({"person": "庄心妍"})
chain2.invoke({"person": "庄心妍", "language": "中文"})

'她出生于中国的福建省福州市。'

在上面的chain2中，chain1的输出作为city的值继续下一条链

</br>

#### **多条链高级使用**

In [9]:
from langchain_core.runnables import RunnablePassthrough

prompt1 = ChatPromptTemplate.from_template(
    "生成一个{attribute}属性的颜色。除了返回这个颜色的名字不要做其他事:"
)
prompt2 = ChatPromptTemplate.from_template(
    "什么水果是这个颜色:{color},只返回这个水果的名字不要做其他事情:"
)
prompt3 = ChatPromptTemplate.from_template(
    "哪个国家的国旗有这个颜色:{color},只返回这个国家的名字不要做其他事情:"
)
prompt4 = ChatPromptTemplate.from_template(
    "有这个颜色的水果是{fruit},有这个颜色的国旗是{country}？"
)


model_parser = model | StrOutputParser()
# 生成一个颜色
color_generator = (
    {"attribute": RunnablePassthrough()} | prompt1 | {"color": model_parser}
)
color_to_fruit = prompt2 | model_parser
color_to_country = prompt3 | model_parser
question_generator = (
    color_generator | {"fruit": color_to_fruit, "country": color_to_country} | prompt4
)

执行顺序：question_generator =  color_generator | {"fruit": color_to_fruit, "country": color_to_country} | prompt4

- 先根据prompt1生成结果，并将结果作为参数color带给后面的链
- color_to_fruit需要color，color_to_country需要的color都在上面传递下来了，执行完成后分别作为参数传递给 fruit 和 country 变量传递给prompt4
- prompt4最终接受fruit和country执行链条生成结果


In [10]:
question_generator.invoke("温柔的")

ChatPromptValue(messages=[HumanMessage(content='有这个颜色的水果是桃子,有这个颜色的国旗是不存在的。？')])

#### 多链执行和结果合并 Branching and Merging

RunnableParallels 允许分割或分叉链，以便多个组件可以并行处理输入。

最后，其他组件可以加入或合并结果以合成最终响应。这种类型的链创建的计算图形如下:

In [ ]:
#      Input
#       / \
#      /   \
#  Branch1 Branch2
#      \   /
#       \ /
#       Combine

In [11]:
planner = (
    ChatPromptTemplate.from_template("生成一个关于{input}的论点")
    | ChatOpenAI()
    | StrOutputParser()
    | {"base_response": RunnablePassthrough()} # RunnablePassthrough将上面结果透传下去
)

arguments_for = (
    ChatPromptTemplate.from_template(
        "列举出以下内容的优点和或积极方面{base_response}"
    )
    | ChatOpenAI()
    | StrOutputParser()
)
arguments_against = (
    ChatPromptTemplate.from_template(
        "列举出以下内容的缺点和或消极方面 {base_response}"
    )
    | ChatOpenAI()
    | StrOutputParser()
)

final_responder = (
    ChatPromptTemplate.from_messages(
        [
            ("ai", "{original_response}"),
            ("human", "积极:\n{results_1}\n\消极:\n{results_2}"),
            ("system", "根据评论给出最后的回复"),
        ]
    )
    | ChatOpenAI()
    | StrOutputParser()
)

chain = (
    planner
    | {
        "results_1": arguments_for,
        "results_2": arguments_against,
        "original_response": itemgetter("base_response"),
    }
    | final_responder
)

In [12]:
chain.invoke({"input": "比特币"})

'比特币作为一种数字货币，的确具有许多积极特性，如去中心化、不可篡改性、有限发行量等，有助于推动金融自由化和国际贸易。然而，也需要认识到比特币存在波动性大、安全性问题、隐私性问题等负面影响。在使用比特币时，需要谨慎考虑这些风险因素，并寻找合适的风险管理策略。比特币的发展和应用仍在不断演变中，我们需要持续关注其发展动态，并做出明智的投资和使用决策。'

### Memory

RunnableWithMessageHistory 允许我们将消息历史添加到某些类型的链中。它包装另一个 Runnable 并为其管理聊天消息历史记录

具体来说，它可用于以下任何输入类型的 Runnable：

- 一个 BaseMessage 序列
- 一个以序列 BaseMessage 为值的字典
- 一个以字符串或序列 BaseMessage 为最新消息的键和以历史消息为值的键的字典


并且返回以下任何输出类型之一：

- 一个可以作为 AIMessage 内容处理的字符串

- 一个 BaseMessage 序列

- 一个包含 BaseMessage 序列的字典


In [13]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai.chat_models import ChatOpenAI

model = ChatOpenAI()
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "你是一个擅长{ability}的助手。回答不超过20个字",
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)
runnable = prompt | model

使用内存存储

In [14]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(
    runnable,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
)

In [15]:
with_message_history.invoke(
    {"ability": "数学", "input": "余弦函数是什么意思？"},
    config={"configurable": {"session_id": "abc123"}},
)

Parent run c5fa54b3-8bdc-4c04-a2a3-8be2a567e8e6 not found for run 2b7022f2-5e94-4772-9678-74a30b66837c. Treating as a root run.


AIMessage(content='表示角的余弦值，介于-1到1之间。', response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 45, 'total_tokens': 62}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-a8653b57-7cda-485c-bc54-57a10c196787-0')

In [16]:
# 记住
with_message_history.invoke(
    {"ability": "数学", "input": "什么？"},
    config={"configurable": {"session_id": "abc123"}},
)

Parent run cb5c8e82-d6eb-4c17-85f2-714b461e31c0 not found for run 30e95118-ac47-42f1-8c29-69f92fc9c723. Treating as a root run.


AIMessage(content='描述角度的余弦值在-1到1之间变化。', response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 74, 'total_tokens': 92}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-a635dd91-55e3-48a2-8795-858fe2a282bb-0')

In [17]:
# 新的 session_id --> 不记住。
with_message_history.invoke(
    {"ability": "数学", "input": "什么？"},
    config={"configurable": {"session_id": "def234"}},
)

Parent run d7e9f1b1-f866-4136-97dc-ce8f60c278c3 not found for run c7341298-6662-45df-95a3-441fd807663e. Treating as a root run.


AIMessage(content='我是一个擅长数学的助手，有什么可以帮到您的吗？', response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 38, 'total_tokens': 65}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-7a2cd72b-bfd8-4f72-a8fc-70349b5b83c2-0')